In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from my_functions import integer_perturbation, get_X_pca_totals, contains_at_least_one, add_new_elements_pca

In [2]:
df = pd.read_excel('data/IncidentiModificato.xlsx', index_col='VERBALE')
df = df.drop(columns=["DATA"])
X = df[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'BMI', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y = df['Mezzo']

df.head()

,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,Testa:Cervelletto,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,0,81,84.0,1.75,27.428571,0,1,0,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,1,69,69.0,1.62,26.291724,1,4,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,1,71,67.0,1.55,27.887617,1,2,0,1,1,...,0,0,0,0,4,6,0,0,4,10
86220,1,54,60.0,1.59,23.733238,1,4,0,0,1,...,0,0,0,0,4,5,3,2,4,14
86247,1,78,69.0,1.67,24.740937,1,2,0,0,0,...,0,0,0,0,4,2,0,2,4,8


Lascio da parte 30 elementi random del dataframe e faccio data augmentation (fino a 1000) con i restanti 100

In [8]:
from sklearn.manifold import TSNE 

X_pca = get_X_pca_totals(df)
random_index = np.random.choice(X_pca.index, 30, False)
X_os, y_os = add_new_elements_pca(df=df.drop(random_index), n_elements=1000)

Traino la rete con i dati aumentati, si ottiene un best score molto vicino a 1

In [12]:
X_os_std = StandardScaler().fit_transform(X_os)

params = {'activation': ['relu', 'logistic', 'tanh', 'normal'],
          'hidden_layer_sizes' : [(2), (3,3), (5), (6,3), (5,5),(6), (8), (10), (20)],
          #'alpha' : [0.0001, 0.001, 0.01, 0.1]
         }

mlp = MLPClassifier(learning_rate_init=0.001, max_iter=5000, learning_rate="adaptive")
clf = GridSearchCV(mlp, params, n_jobs=-1, cv=5)
clf.fit(X_os_std,y_os)
clf.best_score_

0.998

Provo a predire i dati che avevo lasciato da parte ma lo score è basso

In [13]:
X_excluded_std = StandardScaler().fit_transform(X_pca.loc[random_index])
y_excluded = y.loc[random_index]
clf.score(X_excluded_std, y_excluded)

0.6333333333333333